In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import os
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# 1. Load Data
df = pd.read_csv('/content/drive/MyDrive/MachineLearning/Salary_Predict/Salary_Data.csv')

# 2. Drop rows with critical NaNs
df.dropna(subset=["Gender", "Education Level", "Job Title", "Salary"], inplace=True)

# 3. Standardize Education Level
education_mapping = {
    "phD": "PhD",
    "Ph.D": "PhD",
    "Master's Degree": "Master's",
    "Bachelor's Degree": "Bachelor's"
}
df["Education Level"] = df["Education Level"].replace(education_mapping)

# 4. Ordinal Encode Education
edu_encoder = OrdinalEncoder(categories=[["High School", "Bachelor's", "Master's", "PhD"]])
df[["Education Level"]] = edu_encoder.fit_transform(df[["Education Level"]])

# 5. Binary Encode Gender
df["Gender"] = df["Gender"].map({"Male": 1, "Female": 0})

# 6. Save job titles
job_titles = df["Job Title"].dropna().unique().tolist()
os.makedirs("Models", exist_ok=True)
joblib.dump(job_titles, "Models/SP_job_titles.pkl")

# 7. One-hot Encode Job Titles
df = pd.get_dummies(df, columns=["Job Title"], drop_first=True)

# 8. Drop remaining NaNs if any (backup check)
df.dropna(inplace=True)

# 9. Split into features/target
X = df.drop("Salary", axis=1)
y = df["Salary"]

# 10. Split Train-Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 11. Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 12. Train KNN Model
model = KNeighborsRegressor(n_neighbors=1)
model.fit(X_train_scaled, y_train)

# 13. Save Artifacts
joblib.dump(model, "Models/SP_model.pkl")
joblib.dump(scaler, "Models/SP_scaler.pkl")
joblib.dump(X.columns.tolist(), "Models/SP_columns.pkl")
joblib.dump(edu_encoder, "Models/SP_edu_encoder.pkl")

# 14. Evaluate
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"✅ Model Trained")
print(f"🔍 MAE: ${mae:,.2f}")
print(f"📈 R² Score: {r2:.2f}")
